In [86]:
import numpy

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector

In [87]:
# Étape 1 : créer le circuit qui génère la paire intriquée (|ψ->)
def create_entangled_pair_circuit() -> QuantumCircuit:
    circuit = QuantumCircuit(2, 2)

    circuit.x(1)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.z(0)

    return circuit

In [88]:
# Étape 2 : créer 4 nouveaux circuits de mesure
def create_measurement_circuit(angle_a_deg: float, angle_b_deg: float) -> QuantumCircuit:
    qc = QuantumCircuit(2, 2)

    theta_a = numpy.deg2rad(2 * angle_a_deg)
    theta_b = numpy.deg2rad(2 * angle_b_deg)

    qc.ry(theta_a, 0)
    qc.ry(theta_b, 1)
    qc.measure(0, 0)
    qc.measure(1, 1)

    return qc

def create_all_measurement_circuits() -> dict[str, QuantumCircuit]:
    measurement_circuits = {}

    measurement_circuits["(0,0)"] = create_measurement_circuit(0, 0)
    measurement_circuits["(-30,0)"] = create_measurement_circuit(-30, 0)
    measurement_circuits["(0,30)"] = create_measurement_circuit(0, 30)
    measurement_circuits["(-30,30)"] = create_measurement_circuit(-30, 30)

    return measurement_circuits

In [89]:
# Étape 3 : générer deux chaînes de bits aléatoires de taille 1024
def generate_random_measurement_settings(batch_size: int, seed: int | None = None) -> tuple[list[int], list[int]]:
    random_nb = numpy.random.default_rng(seed)

    alice_settings = random_nb.integers(low = 0, high = 2, size = batch_size).tolist()
    bob_settings = random_nb.integers(low = 0, high = 2, size = batch_size).tolist()

    return alice_settings, bob_settings

def generate_step3_settings() -> tuple[list[int], list[int]]:
    batch_size = 1024
    alice_settings, bob_settings = generate_random_measurement_settings(batch_size=batch_size)

    return alice_settings, bob_settings

In [ ]:
# Étape 4 : répéter 1024 fois le processus de mesure avec des circuits choisis aléatoirement
def choose_measurement_circuit_randomly(measurement_circuits: list[QuantumCircuit],rng: numpy.random.Generator) -> QuantumCircuit:
    index = int(rng.integers(low=0, high=len(measurement_circuits)))
    return measurement_circuits[index]

def compose_and_run_once(sampler: StatevectorSampler, entangling_circuit: QuantumCircuit, measurement_circuit: QuantumCircuit):
    full_circuit = entangling_circuit.compose(measurement_circuit, inplace=False)
    job = sampler.run([full_circuit], shots=1)
    return job.result()


def run_step4_random_batch(entangling_circuit: QuantumCircuit, measurement_circuits: list[QuantumCircuit], batch_size: int = 1024,  seed: int | None = None) -> list:
    rng = numpy.random.default_rng(seed)
    sampler = StatevectorSampler()

    results = []

    for _ in range(batch_size):
        chosen_measurement_circuit = choose_measurement_circuit_randomly(measurement_circuits, rng)
        result = compose_and_run_once(sampler, entangling_circuit, chosen_measurement_circuit)
        results.append(result)

    return results

In [ ]:
# Étape 5 : sauvegarder la clé secrète d'Alice
def extract_alice_bit_from_bitstring(bitstring: str) -> int:
    return int(bitstring[1])

def extract_key_from_results(results: list[dict]) -> list[int]:
    key = []

    for result in results:
        if result["measurement_label"] == "(0,0)":
            alice_bit = extract_alice_bit_from_bitstring(result["bitstring"])
            key.append(alice_bit)

    return key

In [ ]:
# Étape 6 :